In [1]:
import glob
import os
import pyart
import numpy as np
import pandas as pd
from copy import deepcopy
import matplotlib.pyplot as plt
%matplotlib inline


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
# suppress anoying iypthon warnings. Not ideal since we suppress also potentially relevant warnings
import warnings
warnings.filterwarnings('ignore')

/opt/anaconda3/envs/exts-ml2/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
# Function to read original dataset
# data is stored as (nz, ny, nx), we return (nx, ny)
def read_nc(fname):
    sat_grid = pyart.io.read_grid(fname)
    for field_name in sat_grid.fields.keys():
        data = np.transpose(np.squeeze(sat_grid.fields[field_name]['data']))
    return data   

In [23]:
def find_min_max(fbasepath, feature):
    flist = glob.glob(fbasepath+'*/NETCDF/'+feature+'/'+'*.nc')
    if len(flist) == 0:
        return None
    flist.sort()
    
    vmin = None
    vmax = None
    for fname in flist:
        # Get time step
        bfile = os.path.basename(fname)
        dt_str = bfile[0:14]
        print(dt_str, end="\r", flush=True)
        
        data = read_nc(fname)
        vmin_aux = np.nanmin(data)
        vmax_aux = np.nanmax(data)
        
        if vmin is None:
            vmin = vmin_aux
        elif vmin > vmin_aux:
            vmin = vmin_aux
        
        if vmax is None:
            vmax = vmax_aux
        elif vmax < vmax_aux:
            vmax = vmax_aux
            
    return vmin, vmax

In [8]:
fbasepath = '/data/pyrad_products/MSG_ML/'
features = ['HRV', 'HRV_norm', 'HRV_text', 'HRV_norm_text', 'IR_108', 'IR_108_text', 'WV_062-IR_108', 'WV_062-IR_108_text', 'POH90']

In [10]:
find_min_max(fbasepath, 'HRV_norm_text')

(0.01230484541923487, 93.56018159483804)

In [24]:
find_min_max(fbasepath, 'IR_108_text')

(0.0, 38.84166709941552)

In [12]:
find_min_max(fbasepath, 'WV_062-IR_108_text')

(2.9200193199910854e-07, 29.53308816169922)